# Advanced Relationships with SQLAlchemy

## Overview

This topic covers advanced relationship patterns in SQLAlchemy, including complex many-to-many relationships, self-referential relationships, polymorphic associations, and hybrid properties. You'll learn how to model sophisticated data relationships and optimize their performance.

## Learning Objectives

By the end of this topic, you will be able to:

1. **Master complex many-to-many relationships** - with additional attributes and custom association objects
2. **Handle self-referential relationships** - hierarchical data, tree structures, and recursive queries
3. **Implement polymorphic associations** - single table inheritance and joined table inheritance
4. **Use hybrid properties** - database-level and Python-level computed properties
5. **Optimize relationship loading** - advanced eager loading strategies and lazy loading patterns

## Prerequisites

- Complete understanding of basic SQLAlchemy relationships
- Familiarity with join concepts
- Knowledge of database normalization
- Understanding of object-oriented programming concepts

Let's explore advanced relationship patterns!


## 1. Setup and Model Definition

Let's create a comprehensive project management system to demonstrate advanced relationship patterns:


In [ ]:
# Setup and model definition for advanced relationships
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text, Boolean, Float, Index, Date, Enum, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, joinedload, subqueryload, selectinload
from sqlalchemy import func, and_, or_, not_, desc, asc, case, cast, extract, distinct
from sqlalchemy.sql import text
from sqlalchemy.ext.hybrid import hybrid_property, hybrid_method
from datetime import datetime, date, timedelta
import enum

# Create database engine
engine = create_engine('sqlite:///advanced_relationships.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

# Enums for better data integrity
class ProjectStatus(enum.Enum):
    PLANNING = "planning"
    ACTIVE = "active"
    ON_HOLD = "on_hold"
    COMPLETED = "completed"
    CANCELLED = "cancelled"

class TaskStatus(enum.Enum):
    TODO = "todo"
    IN_PROGRESS = "in_progress"
    REVIEW = "review"
    DONE = "done"
    BLOCKED = "blocked"

class EmployeeRole(enum.Enum):
    MANAGER = "manager"
    DEVELOPER = "developer"
    DESIGNER = "designer"
    ANALYST = "analyst"
    TESTER = "tester"

print("✅ Setup complete! Now let's explore advanced relationship patterns.")


## 2. Complex Many-to-Many Relationships with Association Objects

When you need to store additional data about the relationship itself, use association objects instead of simple association tables.


In [ ]:
# Association objects for many-to-many relationships with additional attributes
class ProjectEmployee(Base):
    __tablename__ = 'project_employees'
    
    id = Column(Integer, primary_key=True)
    project_id = Column(Integer, ForeignKey('projects.id'), nullable=False)
    employee_id = Column(Integer, ForeignKey('employees.id'), nullable=False)
    role = Column(Enum(EmployeeRole), nullable=False)
    start_date = Column(Date, nullable=False)
    end_date = Column(Date)
    hours_allocated = Column(Float, default=0.0)
    hourly_rate = Column(Float, default=0.0)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    project = relationship("Project", back_populates="project_employees")
    employee = relationship("Employee", back_populates="project_employees")
    
    @hybrid_property
    def total_cost(self):
        if self.hours_allocated and self.hourly_rate:
            return self.hours_allocated * self.hourly_rate
        return 0.0
    
    def __repr__(self):
        return f"<ProjectEmployee(project_id={self.project_id}, employee_id={self.employee_id}, role={self.role.value})>"

print("✅ Association object created with additional attributes!")
print("Features: role, start_date, end_date, hours_allocated, hourly_rate, total_cost calculation")


## 3. Self-Referential Relationships

Self-referential relationships allow a model to reference itself, useful for hierarchical data like organizational structures or task dependencies.


In [ ]:
# Self-referential relationships for hierarchical data
class Employee(Base):
    __tablename__ = 'employees'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50), nullable=False, index=True)
    last_name = Column(String(50), nullable=False, index=True)
    email = Column(String(100), unique=True, nullable=False, index=True)
    phone = Column(String(20))
    hire_date = Column(Date, nullable=False, index=True)
    salary = Column(Float, nullable=False)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    company_id = Column(Integer, ForeignKey('companies.id'), nullable=False)
    department_id = Column(Integer, ForeignKey('departments.id'))
    manager_id = Column(Integer, ForeignKey('employees.id'))  # Self-referential
    
    # Relationships
    company = relationship("Company", back_populates="employees")
    department = relationship("Department", back_populates="employees", foreign_keys=[department_id])
    manager = relationship("Employee", remote_side=[id], backref="subordinates")
    project_employees = relationship("ProjectEmployee", back_populates="employee", cascade="all, delete-orphan")
    assigned_tasks = relationship("Task", back_populates="assignee", cascade="all, delete-orphan")
    
    @hybrid_property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    @hybrid_property
    def years_of_service(self):
        if self.hire_date:
            today = date.today()
            return today.year - self.hire_date.year - ((today.month, today.day) < (self.hire_date.month, self.hire_date.day))
        return 0
    
    @hybrid_method
    def is_manager(self):
        return len(self.subordinates) > 0
    
    def __repr__(self):
        return f"<Employee(name='{self.full_name}', email='{self.email}')>"

print("✅ Self-referential relationship created!")
print("Features: manager-subordinate hierarchy, hybrid properties, years of service calculation")


## 4. Hybrid Properties and Methods

Hybrid properties provide both Python-level and database-level computed attributes, allowing for efficient querying and computation.


In [ ]:
# Hybrid properties for computed attributes
class Project(Base):
    __tablename__ = 'projects'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(200), nullable=False, index=True)
    description = Column(Text)
    start_date = Column(Date, nullable=False, index=True)
    end_date = Column(Date, index=True)
    budget = Column(Float, default=0.0)
    status = Column(Enum(ProjectStatus), default=ProjectStatus.PLANNING, index=True)
    priority = Column(Integer, default=1)  # 1=highest, 5=lowest
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    company_id = Column(Integer, ForeignKey('companies.id'), nullable=False)
    department_id = Column(Integer, ForeignKey('departments.id'))
    project_manager_id = Column(Integer, ForeignKey('employees.id'))
    
    # Relationships
    company = relationship("Company", back_populates="projects")
    department = relationship("Department", back_populates="projects")
    project_manager = relationship("Employee", foreign_keys=[project_manager_id])
    project_employees = relationship("ProjectEmployee", back_populates="project", cascade="all, delete-orphan")
    tasks = relationship("Task", back_populates="project", cascade="all, delete-orphan")
    
    @hybrid_property
    def duration_days(self):
        if self.start_date and self.end_date:
            return (self.end_date - self.start_date).days
        return 0
    
    @hybrid_property
    def team_size(self):
        return len([pe for pe in self.project_employees if pe.is_active])
    
    @hybrid_property
    def total_cost(self):
        return sum(pe.total_cost for pe in self.project_employees if pe.is_active)
    
    def __repr__(self):
        return f"<Project(name='{self.name}', status='{self.status.value}')>"

print("✅ Hybrid properties implemented!")
print("Features: duration_days, team_size, total_cost - all computed dynamically")


## 5. Advanced Relationship Loading Strategies

Optimize relationship loading with different strategies to avoid N+1 problems and improve performance.


In [ ]:
# Advanced relationship loading strategies
class Task(Base):
    __tablename__ = 'tasks'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(200), nullable=False, index=True)
    description = Column(Text)
    start_date = Column(Date, index=True)
    due_date = Column(Date, index=True)
    completed_date = Column(Date)
    estimated_hours = Column(Float, default=0.0)
    actual_hours = Column(Float, default=0.0)
    status = Column(Enum(TaskStatus), default=TaskStatus.TODO, index=True)
    priority = Column(Integer, default=1)  # 1=highest, 5=lowest
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    project_id = Column(Integer, ForeignKey('projects.id'), nullable=False)
    assignee_id = Column(Integer, ForeignKey('employees.id'))
    parent_id = Column(Integer, ForeignKey('tasks.id'))  # Self-referential for subtasks
    
    # Relationships
    project = relationship("Project", back_populates="tasks")
    assignee = relationship("Employee", back_populates="assigned_tasks")
    parent = relationship("Task", remote_side=[id], backref="subtasks")
    
    @hybrid_property
    def is_overdue(self):
        if self.due_date and self.status != TaskStatus.DONE:
            return date.today() > self.due_date
        return False
    
    @hybrid_property
    def progress_percentage(self):
        if self.estimated_hours and self.actual_hours:
            return min(100, (self.actual_hours / self.estimated_hours) * 100)
        return 0
    
    @hybrid_property
    def is_completed(self):
        return self.status == TaskStatus.DONE
    
    def __repr__(self):
        return f"<Task(name='{self.name}', status='{self.status.value}')>"

# Create remaining models for completeness
class Company(Base):
    __tablename__ = 'companies'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(200), nullable=False, unique=True, index=True)
    address = Column(String(300))
    phone = Column(String(20))
    email = Column(String(100))
    website = Column(String(200))
    founded_year = Column(Integer)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Relationships
    employees = relationship("Employee", back_populates="company", cascade="all, delete-orphan")
    projects = relationship("Project", back_populates="company", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Company(name='{self.name}')>"

class Department(Base):
    __tablename__ = 'departments'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, index=True)
    description = Column(Text)
    budget = Column(Float, default=0.0)
    is_active = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign keys
    company_id = Column(Integer, ForeignKey('companies.id'), nullable=False)
    manager_id = Column(Integer, ForeignKey('employees.id'))
    parent_id = Column(Integer, ForeignKey('departments.id'))  # Self-referential
    
    # Relationships
    company = relationship("Company", back_populates="departments")
    manager = relationship("Employee", foreign_keys=[manager_id])
    parent = relationship("Department", remote_side=[id], backref="children")
    employees = relationship("Employee", back_populates="department", foreign_keys="Employee.department_id")
    projects = relationship("Project", back_populates="department", cascade="all, delete-orphan")
    
    @hybrid_property
    def employee_count(self):
        return len(self.employees)
    
    def __repr__(self):
        return f"<Department(name='{self.name}')>"

# Create indexes for performance
Index('idx_employees_name_email', Employee.first_name, Employee.last_name, Employee.email)
Index('idx_projects_name_status', Project.name, Project.status)
Index('idx_tasks_name_status', Task.name, Task.status)
Index('idx_project_employees_project_employee', ProjectEmployee.project_id, ProjectEmployee.employee_id)

# Create tables
Base.metadata.create_all(engine)

print("✅ All advanced relationship models created successfully!")
print("Models: Company, Department, Employee, Project, Task, ProjectEmployee")
print("Features: Complex many-to-many with association objects, self-referential relationships, hybrid properties")


## 6. Working with Complex Relationships

Let's create sample data and demonstrate how to work with these advanced relationship patterns.


In [ ]:
# Create sample data to demonstrate advanced relationships
def create_sample_data():
    """Create comprehensive sample data for advanced relationship demonstrations"""
    session = Session()
    
    # Create companies
    companies = [
        Company(name="TechCorp Solutions", address="123 Tech Street", phone="555-0101", 
                email="info@techcorp.com", website="www.techcorp.com", founded_year=2010),
        Company(name="Innovation Labs", address="456 Innovation Ave", phone="555-0202", 
                email="contact@innovationlabs.com", website="www.innovationlabs.com", founded_year=2015)
    ]
    session.add_all(companies)
    session.commit()
    
    # Create departments with hierarchical structure
    departments = [
        Department(name="Engineering", description="Software development and engineering", 
                  budget=500000, company_id=1),
        Department(name="Frontend Team", description="Frontend development", 
                  budget=200000, company_id=1, parent_id=1),
        Department(name="Backend Team", description="Backend development", 
                  budget=300000, company_id=1, parent_id=1),
        Department(name="Research & Development", description="R&D and innovation", 
                  budget=400000, company_id=2),
        Department(name="Product Management", description="Product strategy and management", 
                  budget=250000, company_id=1)
    ]
    session.add_all(departments)
    session.commit()
    
    # Create employees with hierarchical structure
    employees = [
        # TechCorp employees
        Employee(first_name="Alice", last_name="Johnson", email="alice@techcorp.com", 
                phone="555-1001", hire_date=date(2020, 1, 15), salary=95000, 
                company_id=1, department_id=1),  # Engineering Manager
        Employee(first_name="Bob", last_name="Smith", email="bob@techcorp.com", 
                phone="555-1002", hire_date=date(2021, 3, 10), salary=75000, 
                company_id=1, department_id=2, manager_id=1),  # Frontend Developer
        Employee(first_name="Carol", last_name="Davis", email="carol@techcorp.com", 
                phone="555-1003", hire_date=date(2021, 6, 20), salary=80000, 
                company_id=1, department_id=3, manager_id=1),  # Backend Developer
        Employee(first_name="David", last_name="Wilson", email="david@techcorp.com", 
                phone="555-1004", hire_date=date(2022, 2, 1), salary=70000, 
                company_id=1, department_id=2, manager_id=2),  # Junior Frontend Dev
        Employee(first_name="Eve", last_name="Brown", email="eve@techcorp.com", 
                phone="555-1005", hire_date=date(2020, 8, 15), salary=85000, 
                company_id=1, department_id=5),  # Product Manager
        
        # Innovation Labs employees
        Employee(first_name="Frank", last_name="Miller", email="frank@innovationlabs.com", 
                phone="555-2001", hire_date=date(2019, 4, 1), salary=100000, 
                company_id=2, department_id=4),  # R&D Director
        Employee(first_name="Grace", last_name="Taylor", email="grace@innovationlabs.com", 
                phone="555-2002", hire_date=date(2021, 9, 10), salary=78000, 
                company_id=2, department_id=4, manager_id=6)  # Research Scientist
    ]
    session.add_all(employees)
    session.commit()
    
    # Update department managers
    departments[0].manager_id = 1  # Engineering
    departments[1].manager_id = 2  # Frontend Team
    departments[2].manager_id = 3  # Backend Team
    departments[3].manager_id = 6  # R&D
    departments[4].manager_id = 5  # Product Management
    session.commit()
    
    # Create projects
    projects = [
        Project(name="E-commerce Platform", description="Modern e-commerce solution", 
               start_date=date(2023, 1, 1), end_date=date(2023, 12, 31), 
               budget=500000, status=ProjectStatus.ACTIVE, priority=1, 
               company_id=1, department_id=1, project_manager_id=1),
        Project(name="Mobile App Redesign", description="Redesign mobile application", 
               start_date=date(2023, 3, 1), end_date=date(2023, 9, 30), 
               budget=200000, status=ProjectStatus.ACTIVE, priority=2, 
               company_id=1, department_id=2, project_manager_id=2),
        Project(name="API Optimization", description="Optimize backend API performance", 
               start_date=date(2023, 2, 15), end_date=date(2023, 8, 15), 
               budget=150000, status=ProjectStatus.ACTIVE, priority=3, 
               company_id=1, department_id=3, project_manager_id=3),
        Project(name="AI Research Project", description="Machine learning research initiative", 
               start_date=date(2023, 1, 15), end_date=date(2024, 6, 30), 
               budget=800000, status=ProjectStatus.ACTIVE, priority=1, 
               company_id=2, department_id=4, project_manager_id=6)
    ]
    session.add_all(projects)
    session.commit()
    
    # Create project-employee associations with additional attributes
    project_employees = [
        ProjectEmployee(project_id=1, employee_id=1, role=EmployeeRole.MANAGER, 
                       start_date=date(2023, 1, 1), hours_allocated=40, hourly_rate=50),
        ProjectEmployee(project_id=1, employee_id=2, role=EmployeeRole.DEVELOPER, 
                       start_date=date(2023, 1, 15), hours_allocated=35, hourly_rate=40),
        ProjectEmployee(project_id=1, employee_id=3, role=EmployeeRole.DEVELOPER, 
                       start_date=date(2023, 1, 15), hours_allocated=35, hourly_rate=45),
        ProjectEmployee(project_id=1, employee_id=5, role=EmployeeRole.ANALYST, 
                       start_date=date(2023, 1, 1), hours_allocated=20, hourly_rate=55),
        
        ProjectEmployee(project_id=2, employee_id=2, role=EmployeeRole.DEVELOPER, 
                       start_date=date(2023, 3, 1), hours_allocated=40, hourly_rate=40),
        ProjectEmployee(project_id=2, employee_id=4, role=EmployeeRole.DEVELOPER, 
                       start_date=date(2023, 3, 15), hours_allocated=30, hourly_rate=35),
        
        ProjectEmployee(project_id=3, employee_id=3, role=EmployeeRole.DEVELOPER, 
                       start_date=date(2023, 2, 15), hours_allocated=40, hourly_rate=45),
        
        ProjectEmployee(project_id=4, employee_id=6, role=EmployeeRole.MANAGER, 
                       start_date=date(2023, 1, 15), hours_allocated=40, hourly_rate=60),
        ProjectEmployee(project_id=4, employee_id=7, role=EmployeeRole.ANALYST, 
                       start_date=date(2023, 2, 1), hours_allocated=35, hourly_rate=50)
    ]
    session.add_all(project_employees)
    session.commit()
    
    # Create tasks with hierarchical structure
    tasks = [
        # E-commerce Platform tasks
        Task(name="Database Design", description="Design database schema", 
             start_date=date(2023, 1, 1), due_date=date(2023, 2, 15), 
             estimated_hours=80, status=TaskStatus.DONE, priority=1, 
             project_id=1, assignee_id=3),
        Task(name="User Authentication", description="Implement user authentication system", 
             start_date=date(2023, 1, 15), due_date=date(2023, 3, 1), 
             estimated_hours=120, status=TaskStatus.IN_PROGRESS, priority=1, 
             project_id=1, assignee_id=3),
        Task(name="Frontend Components", description="Create reusable UI components", 
             start_date=date(2023, 2, 1), due_date=date(2023, 4, 1), 
             estimated_hours=160, status=TaskStatus.IN_PROGRESS, priority=2, 
             project_id=1, assignee_id=2),
        Task(name="Payment Integration", description="Integrate payment gateway", 
             start_date=date(2023, 3, 1), due_date=date(2023, 5, 1), 
             estimated_hours=100, status=TaskStatus.TODO, priority=2, 
             project_id=1, assignee_id=3, parent_id=2),  # Subtask of User Authentication
        
        # Mobile App Redesign tasks
        Task(name="UI/UX Research", description="Research user interface patterns", 
             start_date=date(2023, 3, 1), due_date=date(2023, 4, 15), 
             estimated_hours=60, status=TaskStatus.DONE, priority=1, 
             project_id=2, assignee_id=2),
        Task(name="Component Library", description="Build component library", 
             start_date=date(2023, 4, 1), due_date=date(2023, 6, 30), 
             estimated_hours=200, status=TaskStatus.IN_PROGRESS, priority=1, 
             project_id=2, assignee_id=2),
        Task(name="Testing Framework", description="Set up testing framework", 
             start_date=date(2023, 4, 15), due_date=date(2023, 5, 15), 
             estimated_hours=80, status=TaskStatus.TODO, priority=2, 
             project_id=2, assignee_id=4, parent_id=6),  # Subtask of Component Library
        
        # AI Research Project tasks
        Task(name="Literature Review", description="Review existing research papers", 
             start_date=date(2023, 1, 15), due_date=date(2023, 3, 31), 
             estimated_hours=120, status=TaskStatus.DONE, priority=1, 
             project_id=4, assignee_id=7),
        Task(name="Data Collection", description="Collect training datasets", 
             start_date=date(2023, 2, 1), due_date=date(2023, 6, 30), 
             estimated_hours=300, status=TaskStatus.IN_PROGRESS, priority=1, 
             project_id=4, assignee_id=7),
        Task(name="Model Development", description="Develop machine learning models", 
             start_date=date(2023, 4, 1), due_date=date(2024, 2, 28), 
             estimated_hours=800, status=TaskStatus.TODO, priority=1, 
             project_id=4, assignee_id=6)
    ]
    session.add_all(tasks)
    session.commit()
    
    print("✅ Comprehensive sample data created!")
    print(f"Companies: {len(companies)}")
    print(f"Departments: {len(departments)} (with hierarchical structure)")
    print(f"Employees: {len(employees)} (with manager-subordinate relationships)")
    print(f"Projects: {len(projects)}")
    print(f"Project-Employee associations: {len(project_employees)} (with roles and costs)")
    print(f"Tasks: {len(tasks)} (with hierarchical subtasks)")
    
    session.close()

# Create the sample data
create_sample_data()


## 7. Advanced Relationship Queries

Now let's explore complex queries that leverage these advanced relationship patterns.


In [ ]:
# Advanced relationship queries demonstration
session = Session()

print("=== ADVANCED RELATIONSHIP QUERIES ===\n")

# 1. Query employees with their managers and subordinates
print("1. Employee Hierarchy (Managers and Subordinates):")
employees_with_hierarchy = session.query(Employee).options(
    joinedload(Employee.manager),
    joinedload(Employee.subordinates)
).all()

for emp in employees_with_hierarchy:
    manager_name = emp.manager.full_name if emp.manager else "No Manager"
    subordinate_count = len(emp.subordinates)
    print(f"  {emp.full_name} (Manager: {manager_name}, Subordinates: {subordinate_count})")

print("\n" + "="*60 + "\n")

# 2. Query projects with their team members and roles
print("2. Projects with Team Members and Roles:")
projects_with_teams = session.query(Project).options(
    joinedload(Project.project_employees).joinedload(ProjectEmployee.employee)
).all()

for project in projects_with_teams:
    print(f"  Project: {project.name}")
    print(f"    Duration: {project.duration_days} days")
    print(f"    Team Size: {project.team_size}")
    print(f"    Total Cost: ${project.total_cost:,.2f}")
    print("    Team Members:")
    for pe in project.project_employees:
        if pe.is_active:
            print(f"      - {pe.employee.full_name} ({pe.role.value}) - ${pe.total_cost:,.2f}")
    print()

print("="*60 + "\n")

# 3. Query departments with hierarchical structure
print("3. Department Hierarchy:")
departments_with_hierarchy = session.query(Department).options(
    joinedload(Department.parent),
    joinedload(Department.children),
    joinedload(Department.manager)
).all()

def print_department_tree(dept, level=0):
    indent = "  " * level
    manager_name = dept.manager.full_name if dept.manager else "No Manager"
    print(f"{indent}📁 {dept.name} (Manager: {manager_name}, Budget: ${dept.budget:,.0f})")
    for child in dept.children:
        print_department_tree(child, level + 1)

# Print root departments (those without parents)
root_departments = [d for d in departments_with_hierarchy if d.parent is None]
for dept in root_departments:
    print_department_tree(dept)

print("\n" + "="*60 + "\n")

# 4. Query tasks with hierarchical structure and progress
print("4. Task Hierarchy with Progress:")
tasks_with_hierarchy = session.query(Task).options(
    joinedload(Task.parent),
    joinedload(Task.subtasks),
    joinedload(Task.assignee),
    joinedload(Task.project)
).all()

def print_task_tree(task, level=0):
    indent = "  " * level
    assignee_name = task.assignee.full_name if task.assignee else "Unassigned"
    status_emoji = {"todo": "📋", "in_progress": "🔄", "review": "👀", "done": "✅", "blocked": "🚫"}
    emoji = status_emoji.get(task.status.value, "❓")
    
    print(f"{indent}{emoji} {task.name} ({task.status.value})")
    print(f"{indent}   Assignee: {assignee_name}")
    print(f"{indent}   Progress: {task.progress_percentage:.1f}% ({task.actual_hours}/{task.estimated_hours}h)")
    print(f"{indent}   Overdue: {'Yes' if task.is_overdue else 'No'}")
    
    for subtask in task.subtasks:
        print_task_tree(subtask, level + 1)

# Print root tasks (those without parents)
root_tasks = [t for t in tasks_with_hierarchy if t.parent is None]
for task in root_tasks:
    print_task_tree(task)
    print()

print("="*60 + "\n")

# 5. Complex aggregation queries using hybrid properties
print("5. Project Statistics using Hybrid Properties:")
project_stats = session.query(Project).all()

for project in project_stats:
    print(f"  {project.name}:")
    print(f"    Duration: {project.duration_days} days")
    print(f"    Team Size: {project.team_size} members")
    print(f"    Total Cost: ${project.total_cost:,.2f}")
    print(f"    Budget Utilization: {(project.total_cost/project.budget*100):.1f}%" if project.budget > 0 else "    Budget: Not set")
    print()

print("="*60 + "\n")

# 6. Query employees with their project assignments and costs
print("6. Employee Project Assignments and Costs:")
employees_with_projects = session.query(Employee).options(
    joinedload(Employee.project_employees).joinedload(ProjectEmployee.project)
).all()

for emp in employees_with_projects:
    total_cost = sum(pe.total_cost for pe in emp.project_employees if pe.is_active)
    active_projects = [pe.project.name for pe in emp.project_employees if pe.is_active]
    
    print(f"  {emp.full_name} (Years of Service: {emp.years_of_service})")
    print(f"    Active Projects: {', '.join(active_projects) if active_projects else 'None'}")
    print(f"    Total Project Cost: ${total_cost:,.2f}")
    print(f"    Is Manager: {'Yes' if emp.is_manager() else 'No'}")
    print()

session.close()

print("✅ Advanced relationship queries completed!")
print("Demonstrated: hierarchical queries, association objects, hybrid properties, complex aggregations")


## 8. Best Practices and Common Pitfalls

### Best Practices for Advanced Relationships

1. **Use Association Objects When Needed**: Only use association objects when you need additional attributes on the relationship
2. **Optimize Relationship Loading**: Use appropriate loading strategies (`joinedload`, `subqueryload`, `selectinload`) to avoid N+1 problems
3. **Index Foreign Keys**: Always index foreign key columns for better query performance
4. **Use Hybrid Properties Wisely**: Hybrid properties are powerful but can impact query performance if overused
5. **Handle Circular References**: Be careful with bidirectional relationships to avoid circular import issues

### Common Pitfalls to Avoid

1. **N+1 Query Problems**: Always use eager loading for relationships you know you'll access
2. **Infinite Recursion**: Be careful with self-referential relationships in recursive queries
3. **Memory Issues**: Large hierarchical structures can consume significant memory
4. **Cascade Confusion**: Understand cascade options to avoid unintended deletions
5. **Performance Degradation**: Complex relationships can slow down queries if not properly optimized

### Summary

Advanced relationships in SQLAlchemy provide powerful tools for modeling complex data structures:

- **Association Objects**: Store additional data about many-to-many relationships
- **Self-Referential Relationships**: Model hierarchical data like organizational structures
- **Hybrid Properties**: Create computed attributes that work at both Python and database levels
- **Advanced Loading**: Optimize relationship loading to prevent performance issues

These patterns are essential for building sophisticated applications with complex data relationships while maintaining good performance and code maintainability.
